In [ ]:
# Evaluate Site 3 detections against an input json label file 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
module_path = module_path + '/code/'
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import json 
import renderapi
import scipy
import numpy as np
from scipy.stats import norm
from at_synapse_detection.render_module import RenderModule

from at_synapse_detection import processDetections as pd
import matplotlib
import matplotlib.pyplot as plt
from shapely import geometry
from skimage import measure
from at_synapse_detection import synaptogram
from scipy import io
from at_synapse_detection import SynapseDetection as syn

In [15]:
import importlib
importlib.reload(synaptogram)

<module 'at_synapse_detection.synaptogram' from '/Users/anish/Documents/Connectome/SynapseAnalysis/at_synapse_detection/synaptogram.py'>

In [5]:
args={
    'render':{
    'host':'http://152.3.214.113',
    'port':8080,
    'owner':'Anish',
    'project':'M247514_Rorb_1',
    'client_scripts':'/home/anish/Connectome/render/render-ws-java-client/src/main/scripts'
},
'log_level':'DEBUG'}
mod = RenderModule(input_data = args,args=[])
stacks = mod.render.run(renderapi.render.get_stacks_by_owner_project)
render_args = args['render']

DEBUG:renderapi.render:Render object created with host=http://152.3.214.113, port=8080, project=M247514_Rorb_1, owner=Anish, scripts=/home/anish/Connectome/render/render-ws-java-client/src/main/scripts
INFO:renderapi.render:Client script /home/anish/Connectome/render/render-ws-java-client/src/main/scripts/run_ws_client.sh not found!
DEBUG:renderapi.render:Render object created with host=http://152.3.214.113, port=8080, project=M247514_Rorb_1, owner=Anish, scripts=/home/anish/Connectome/render/render-ws-java-client/src/main/scripts
DEBUG:renderapi.render:http://152.3.214.113:8080/render-ws/v1/owner/Anish/stacks/
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 152.3.214.113
DEBUG:urllib3.connectionpool:http://152.3.214.113:8080 "GET /render-ws/v1/owner/Anish/stacks/ HTTP/1.1" 200 None


In [7]:
metadataFN = '../data/M247514_Rorb_1/Site3Align2/site3_metadata.json'
metadata = syn.loadMetadata(metadataFN)
n = 0 
datalocation = metadata['datalocation']
outputFN = os.path.join(datalocation, 'resultVol')
outputFN = outputFN + str(n) + '.json'

In [9]:
annotationdata = json.load(open('../data/M247514_Rorb_1/Site3Align2/json_annotations/m247514_Site3Annotation_MN_global_v2.json'))


In [10]:
detectiondata = json.load(open(outputFN))

In [11]:
annotationList = annotationdata['area_lists']
detectionList = detectiondata['area_lists']

In [13]:
print(len(annotationList))
print(len(detectionList))

1078
4946


In [30]:
detectionsExpanded = []

itr = 0
for detection in detectionList: 
    bboxCoordinates = synaptogram.getAnnotationBoundingBox2(detection, render_args)
    expandedBox = synaptogram.expandBoundingBox(bboxCoordinates, win_xy=0, win_z=0)
    detectionpoly = pd.createPolygonFromBBox(expandedBox)
    zlistdetection = synaptogram.getZListFromBoundingBox(expandedBox)

    zListAndPoly = {'zlist': zlistdetection, 'poly': detectionpoly}
    detectionsExpanded.append(zListAndPoly)

    

In [31]:
synapsesExpanded = []

for synapse in annotationList: 
    bboxCoordinates = synaptogram.getAnnotationBoundingBox2(synapse, render_args)
    bboxCoordinates = synaptogram.transformSynapseCoordinates(bboxCoordinates)
    expandedBox = synaptogram.expandBoundingBox(bboxCoordinates, 0, 0)
    synapsepoly = pd.createPolygonFromBBox(expandedBox)
    zlistsynapse = synaptogram.getZListFromBoundingBox(expandedBox)

    zListAndPoly = {'zlist': zlistsynapse, 'poly': synapsepoly}
    synapsesExpanded.append(zListAndPoly)

In [32]:
overlapMatrix = np.zeros((len(detectionsExpanded), len(synapsesExpanded)))

cItr = 0
for synapse in synapsesExpanded: 

    zlistSynapse = synapse['zlist']
    polySynapse = synapse['poly']

    rItr = 0 
    for detection in detectionsExpanded: 

        zlistDetection = detection['zlist']
        polydetection = detection['poly']

        if not set(zlistDetection).isdisjoint(zlistSynapse):
            if polySynapse.intersects(polydetection): 
                overlapMatrix[rItr, cItr] = 1

        rItr = rItr + 1
    cItr = cItr + 1
    

In [33]:
synapseOverlap = (np.sum(overlapMatrix, 0))

In [34]:
np.count_nonzero(synapseOverlap) / float(len(synapseOverlap))

0.3450834879406308

In [35]:
np.count_nonzero(synapseOverlap)

372

In [36]:
float(len(synapseOverlap))

1078.0

In [ ]:
# create synaptograms with detections 

In [ ]:
synapse = annotationList[6]
win_xy = 4
win_z = 1
filepath = '/Users/anish/Documents/Connectome/Synaptome-Duke/data/collman17/Site3Align2Stacks/';
stackList = ['results',  'PSD95', 'synapsin', 'VGlut1', 'GluN1', 'GABA', 'Gephyrin']
showProb = True
textXOffset = 0
textYOffset = 5


